# Embedding an AOP NEON dataset in a Jupyter Notebook with Bokeh

This notebook shows access to the AOP datasets from the product <b>DP3.30010.001</b> made availble from the NEON web portal. 
The Jupyter notebook includes direct use of the Bokeh server without need to open a separate webpage.

For the NEON project the user interface went from [sending a harddrivive through this webpage registration](https://www.neonscience.org/aop-data-hard-drive-request) to browsing the data interactively on the [NEON data portal (scroll to the bottom of this webpage)](https://data.neonscience.org/data-products/DP3.30010.001).

[Jupyter notebook](https://jupyter.org/)  created by  [Valerio Pascucci](http://cedmav.com/), [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html), and [Steve Petruzza](https://stevepetruzza.io/)

***
#### This code can be run within a selfcontained Jupyter Notebook or as a webpage.
+ For the first case just continue to execute the following cells to the end.
+ For the second case the you need to execute the following two commendd to (1) convert the notebook to a regular python file and (2) start the Bokeh server to show the interactive webpage.  
<code> 
> jupyter nbconvert --to script neon.ipynb
> python -m bokeh serve  --show neon.py  --port 5012
</code>


In [1]:
import yaml

from bokeh.layouts import row,column
from bokeh.models.widgets import Div
from bokeh.models import ColumnDataSource, Slider , Dropdown, Select
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook, curdoc
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

from IPython.display import clear_output
from IPython.display import IFrame

import requests
from requests.exceptions import HTTPError


In [2]:
def in_notebook():
    from IPython import get_ipython
    if get_ipython():
        return True
    else:
        return False    

In [3]:
ShowWebpage = True

if in_notebook():
    ShowWebpage = False

if ShowWebpage:
    pass
else:
    output_notebook()

Loading BokehJS ...

### Parsing of the metedata of the list of datasets availble obtained with a query to the portal.

In [4]:
jsonResponse = None
product_name = "DP3.30010.001"

try:
    response = requests.get("https://neon.visus.org/neonapi/products/DP3.30010.001")
    response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

NEON_site_codes = []
NEON_dictionary = jsonResponse['data']['siteCodes']
index = 0
for k in NEON_dictionary:
    NEON_site_codes.append(k['siteCode'])

Current_site    = NEON_site_codes[0]
NEON_site_codes = NEON_site_codes
NEON__site = {}

for i in NEON_dictionary:
        NEON__site[i['siteCode']] = {}
        #print (i['availableMonths'])
        for j,k in zip(i['availableMonths'],i['availableDataUrls']) :
            NEON__site[i['siteCode']][j]=k



### Create the Bokeh widgets and interactions.

In [5]:
def bkapp(doc):
    class Current_values:
        iframe_width  = 600
        current_site  = NEON_site_codes[0]
        current_month = list(NEON__site[current_site])[0]
        current_url   = "https://neon.visus.org/visus-frame.html?server="+NEON__site[current_site][current_month][7:]
        
    div=Div(text="<iframe src="+ Current_values.current_url + "  width='"+ str(Current_values.iframe_width) + 
            "' height='"+ str(Current_values.iframe_width) + "'  >< /iframe>")
        
    def reset_month():
        Current_values.current_month = list(NEON__site[Current_values.current_site])[0]
        Current_values.current_url   = "https://neon.visus.org/visus-frame.html?server="+NEON__site[Current_values.current_site][Current_values.current_month][7:]

    def update_iframe():
        div.text="<iframe src="+ Current_values.current_url + "  width='"+ str(Current_values.iframe_width) + "' height='"+ str(Current_values.iframe_width) + "'  ><  /iframe>"

    def callback(attr, old, new):
        Current_values.iframe_width = new
        update_iframe()
        

    slider = Slider(start=0, end=1000, value=Current_values.iframe_width, step=1, title="Iframe width")
    slider.on_change('value', callback)
    
    def onChooseMonth(attr, old, new):
        Current_values.current_month = new
        Current_values.current_url   = "https://neon.visus.org/visus-frame.html?server="+NEON__site[Current_values.current_site][Current_values.current_month][7:]
        update_iframe()
        

    choose_month = Select(title="Choose a month availble for site "+ str(NEON_site_codes[0]) , 
                          options= list(NEON__site[NEON_site_codes[0]].keys()))
    choose_month.on_change('value',onChooseMonth)
    #choose_month.on_click(onChooseUrlChange)
    
    def onChooseSite(attr, old, new):
        #site=event.item
        choose_month.options = list(NEON__site[new].keys())
        Current_values.current_site = new
        reset_month()
        update_iframe()

    select_site = Select(title="Sites availble for procut DP3.30010.001:", 
                         options=NEON_site_codes)
    select_site.on_change('value',onChooseSite)

    doc.add_root(column(select_site,choose_month,slider, div))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

### Run the interactive viewer.

In [6]:
if ShowWebpage:
    bkapp(curdoc())
else:
    show(bkapp)